In [75]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import (
    confusion_matrix, 
    plot_confusion_matrix,
    roc_auc_score,
    recall_score
)

# To identify which weather lag is best

In [76]:
# Please check error and rerun the code in notebook 1
# so that 'trap' is included in all the weather_cleaned_lag files


In [77]:
lag0 = pd.read_csv(r'../data/weather_cleaned_lag0.csv')
lag2 = pd.read_csv(r'../data/weather_cleaned_lag2.csv')
lag3 = pd.read_csv(r'../data/weather_cleaned_lag3.csv')
lag5 = pd.read_csv(r'../data/weather_cleaned_lag5.csv')
lag7 = pd.read_csv(r'../data/weather_cleaned_lag7.csv')
lag14 = pd.read_csv(r'../data/weather_cleaned_lag14.csv')
lag21 = pd.read_csv(r'../data/weather_cleaned_lag21.csv')
lag28 = pd.read_csv(r'../data/weather_cleaned_lag28.csv')

In [4]:
# Extract train from train/test
# recommend to split sets by year as splitting by index is more prone to error

lag7['date'] = pd.to_datetime(lag7['date'])
lag7['year'] = lag7['date'].dt.year
lag7_train = lag7.loc[lag7['year'].isin([2007, 2009, 2011, 2013]), :]

In [5]:
# To drop ['nummosquitos', 'wnvcount',        # not available in test set
#          'station',                         # already associated weather to each station
#          'year',
#          'codesum', 'species']              # will be dummied below

lag7_train.drop(columns = ['nummosquitos','wnvcount',
                           'station', 'year', 'codesum', 'species'],
                axis=1,
                inplace = True)

/var/folders/0t/nymtt02n1zqdc1ksr0n857bc0000gn/T/ipykernel_1434/503540313.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lag7_train.drop(columns = ['nummosquitos','wnvcount',


In [79]:
def rf_weather_lag(df, lag_num, concat_df=None):
    df = df.dropna()
    X = df[[col for col in df.columns if (df[col].dtypes !='O') & 
                      (col not in ['date','block','addressaccuracy','nummosquitos','wnvcount','wnvpresent', 'station','species'])]]
    y = df['wnvpresent']
    
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        stratify=y,
        random_state=42
    )
    
    ss = StandardScaler()
    
    X_train = ss.fit_transform(X_train)
    X_test = ss.transform(X_test)
    
    rf = RandomForestClassifier(n_estimators=100)
    
    train_acc = cross_val_score(
        rf,
        X_train,
        y_train,
        cv=5
    ).mean()
    
    test_acc = cross_val_score(
        rf,
        X_test,
        y_test,
        cv=5
    ).mean()
    
    rf.fit(X_train, y_train)
    
    forest_predictions = rf.predict(X_test)
    
    tn, fp, fn, tp = confusion_matrix(y_test, forest_predictions).ravel()
    
    sensitivity = recall_score(y_test, forest_predictions)

    specificity = (tn / (tn + fp))
    
    roc_auc = roc_auc_score(y_test, rf.predict_proba(X_test)[:,1])
    
    weather_dict = {
        'weather_lag': lag_num, 
        'train_acc': [round(train_acc * 100, 2)], 
        'test_acc': [round(test_acc * 100, 2)], 
        'sensitivity': [round(sensitivity * 100, 2)], 
        'specificity': [round(specificity * 100, 2)],
        'roc_auc_score': [round(roc_auc * 100, 2)]
    }
    
    new_df = pd.DataFrame(weather_dict)
    
    if isinstance(concat_df, pd.DataFrame) == True:
        final_df = pd.concat(
            objs=[concat_df, new_df],
            axis=0
        )
        final_df.reset_index(drop=True, inplace=True)
        return final_df
        
    else:    
        return new_df
    

In [126]:
weather_lag_df = rf_weather_lag(lag0, 0)
weather_lag_df = rf_weather_lag(lag2, 2, weather_lag_df)
weather_lag_df = rf_weather_lag(lag3, 3, weather_lag_df)
weather_lag_df = rf_weather_lag(lag5, 5, weather_lag_df)
weather_lag_df = rf_weather_lag(lag7, 7, weather_lag_df)
weather_lag_df = rf_weather_lag(lag14, 14, weather_lag_df)
weather_lag_df = rf_weather_lag(lag21, 21, weather_lag_df)
weather_lag_df = rf_weather_lag(lag28, 28, weather_lag_df)

In [127]:
weather_lag_df

,weather_lag,train_acc,test_acc,sensitivity,specificity,roc_auc_score
0,0,92.90,93.25,17.54,97.31,77.79
1,2,92.98,93.30,16.67,97.36,75.39
2,3,92.92,93.20,18.42,97.51,77.55
3,5,92.98,93.44,18.42,97.51,75.61
4,7,92.90,93.30,16.67,97.51,75.66
5,14,92.95,93.53,17.54,97.01,76.76
6,21,93.03,93.35,19.30,97.41,77.10
7,28,93.21,93.30,18.42,97.94,75.14


In [5]:
def rf_2_weather_lag(df, lag_num, concat_df=None):
    df = df.dropna()
    X = df[[col for col in df.columns if (df[col].dtypes !='O') & 
                      (col not in ['date','block','addressaccuracy','nummosquitos','wnvcount','wnvpresent', 'station','species'])]]
    y = df['wnvpresent']
    
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        stratify=y,
        random_state=42
    )
    
    ss = StandardScaler()
    
    X_train = ss.fit_transform(X_train)
    X_test = ss.transform(X_test)
    
    
    rf =  RandomForestClassifier(
        n_estimators=1500,
        ccp_alpha=0,
        max_depth=10,
        min_samples_split=5,
        min_samples_leaf=5,
        random_state=42,
        class_weight='balanced_subsample')
    
    train_acc = cross_val_score(
        rf,
        X_train,
        y_train,
        cv=5
    ).mean()
    
    test_acc = cross_val_score(
        rf,
        X_test,
        y_test,
        cv=5
    ).mean()
    
    rf.fit(X_train, y_train)
    
    forest_predictions = rf.predict(X_test)
    
    tn, fp, fn, tp = confusion_matrix(y_test, forest_predictions).ravel()
    
    sensitivity = recall_score(y_test, forest_predictions)

    specificity = (tn / (tn + fp))
    
    roc_auc = roc_auc_score(y_test, rf.predict_proba(X_test)[:,1])
    
    weather_dict = {
        'weather_lag': lag_num, 
        'train_acc': [round(train_acc * 100, 2)], 
        'test_acc': [round(test_acc * 100, 2)], 
        'sensitivity': [round(sensitivity * 100, 2)], 
        'specificity': [round(specificity * 100, 2)],
        'roc_auc_score': [round(roc_auc * 100, 2)]
    }
    
    new_df = pd.DataFrame(weather_dict)
    
    if isinstance(concat_df, pd.DataFrame) == True:
        final_df = pd.concat(
            objs=[concat_df, new_df],
            axis=0
        )
        final_df.reset_index(drop=True, inplace=True)
        return final_df
        
    else:    
        return new_df
    

In [ ]:
weather_lag_df2 = rf_2_weather_lag(lag0, 0)
weather_lag_df2 = rf_2_weather_lag(lag2, 2, weather_lag_df2)
weather_lag_df2 = rf_2_weather_lag(lag3, 3, weather_lag_df2)
weather_lag_df2 = rf_2_weather_lag(lag5, 5, weather_lag_df2)
weather_lag_df2 = rf_2_weather_lag(lag7, 7, weather_lag_df2)
weather_lag_df2 = rf_2_weather_lag(lag14, 14, weather_lag_df2)
weather_lag_df2 = rf_2_weather_lag(lag21, 21, weather_lag_df2)
weather_lag_df2 = rf_2_weather_lag(lag28, 28, weather_lag_df2)

In [ ]:
weather_lag_df2

# Recommended weather lag

In [8]:
# weather lag(##) is best
# to proceed to see which feature combination is best

In [ ]:
lag2 = pd.read_csv(r'../data/weather_cleaned_lag2.csv')

In [ ]:
lag2.columns

In [43]:
lag0.columns

Index(['date', 'species', 'latitude', 'longitude', 'trap', 'nummosquitos',
       'wnvpresent', 'wnvcount', 'station', 'tmax', 'tmin', 'tavg', 'dewpoint',
       'wetbulb', 'heat', 'cool', 'codesum', 'preciptotal', 'stnpressure',
       'sealevel', 'resultspeed', 'resultdir', 'avgspeed', 'month', 'sunrise',
       'sunset', 'bcfg', 'br', 'dz', 'fg', 'fg+', 'fu', 'hz', 'mifg', 'ra',
       'sn', 'sq', 'ts', 'tsra', 'vcts', 'gr', 'vcfg'],
      dtype='object')

# Identify traps which are hotspots

In [6]:
# import data

merge_df = pd.read_csv('../data/merged_train.csv')

# create time related features

merge_df['date']=pd.to_datetime(merge_df['date'])
merge_df['year'] = merge_df['date'].dt.year
merge_df['month'] = merge_df['date'].dt.month
merge_df['week'] = merge_df['date'].dt.isocalendar().week

# subset required columns and rows
betw_year = merge_df[['trap', 'year', 'wnvpresent', 'wnvcount']]
betw_year = betw_year.loc[betw_year['wnvpresent']>0,:]           # only rows where wnv is present

# Collapse by trap and year
betw_year = betw_year.groupby(['trap', 'year'], as_index=False)[['wnvpresent', 'wnvcount']]\
.sum().sort_values(['year','wnvpresent'], ascending=False)

In [7]:
# Is there a consistent wave of WNV in each year?

betw_year.groupby('year')[['wnvpresent']].sum()

,wnvpresent
year,
2007,189.0
2009,19.0
2011,50.0
2013,199.0


No, mosquitos thrive in specific years, probably due to favourable conditions in 2007 and 2013 but not in 2009 and 2011. No need to model these as the effects will be taken care of by the weather variables

In [8]:
# Are mosquitos always found in the same location? 
# IE MOSQUITOS THRIVE IN SPECIFIC LOCATIONS Reardless of year on year changes

betw_year['years_wnvp'] = betw_year['wnvpresent'].map(lambda x: 1 if x >0 else 0)

betw_year.groupby(['trap'])['wnvpresent', 'years_wnvp'].sum()\
.sort_values('years_wnvp', ascending=False)[['years_wnvp']].value_counts()


C:\Users\User\AppData\Local\Temp/ipykernel_24512/1500054444.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  betw_year.groupby(['trap'])['wnvpresent', 'years_wnvp'].sum()\


years_wnvp
1             51
2             26
3             16
4              4
dtype: int64

Not exactly, while there are some traps that have wnvpresent in more than one year, more than half of the traps have wnvpresent only in that year.

In [9]:
inter_year = betw_year.pivot(index='trap', columns='year', values=['wnvpresent'])
inter_year.fillna(0, inplace=True)

In [10]:
# positive traps in 2007
ptrap_2007 = betw_year.loc[betw_year['year']==2007, 'trap']
ptrap_2009 = betw_year.loc[betw_year['year']==2009, 'trap']
ptrap_2011 = betw_year.loc[betw_year['year']==2011, 'trap']
ptrap_2013 = betw_year.loc[betw_year['year']==2013, 'trap']

ptrap_2007 = set(ptrap_2007)
ptrap_2009 = set(ptrap_2009)
ptrap_2011 = set(ptrap_2011)
ptrap_2013 = set(ptrap_2013)

In [11]:
ptrap_2007.intersection(ptrap_2009, ptrap_2011, ptrap_2013)

{'T002', 'T090', 'T095', 'T158'}

In [12]:
ptrap_2007_2013 = ptrap_2007.intersection(ptrap_2013)
len(ptrap_2007_2013)

33

# Groupings

Across years: Appear in 
    at least 3 of 4 years (20 traps) ay_3of4
    at least 2 of 4 years (46 traps) ay_2of4

Across years: wnvpresent 
    > wnvpresent 8 times (17 traps) ay_8nabov
    > wnvpresent 6 times (25 traps) ay_6nabov    

Within years: wnvpresent at least 4
2007 (19 traps) wy_2007_4nabv
2009 (0 traps)  0
2011 (3 traps)  wy_2011_4nabv
2013 (22 traps) wy_2013_4nabv


Within years: wnvpresent at least 3
2007 (27 traps) wy_2007_3nabv
2009 (0 traps)  0
2011 (4 traps)  wy_2011_3nabv
2013 (33 traps) wy_2013_3nabv

# Across years: frequency of appearance across years

Across years: Appear in 
    at least 3 of 4 years (20 traps) ay_3of4
    at least 2 of 4 years (46 traps) ay_2of4

In [13]:
ay = betw_year.groupby(['trap'])['wnvpresent', 'years_wnvp'].sum()\
.sort_values('years_wnvp', ascending=False)
ay[:5]

C:\Users\User\AppData\Local\Temp/ipykernel_24512/2786419457.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ay = betw_year.groupby(['trap'])['wnvpresent', 'years_wnvp'].sum()\


,wnvpresent,years_wnvp
trap,,
T002,15.0,4
T090,7.0,4
T095,5.0,4
T158,5.0,4
T028,9.0,3


In [14]:
ay_3of4 = ay.loc[ay['years_wnvp']>2,:].index
ay_3of4

Index(['T002', 'T090', 'T095', 'T158', 'T028', 'T003', 'T114', 'T115', 'T151',
       'T039', 'T031', 'T073', 'T011', 'T225', 'T900', 'T013', 'T230', 'T008',
       'T221', 'T009'],
      dtype='object', name='trap')

In [15]:
ay_2of4 = ay.loc[ay['years_wnvp']>1,:].index
ay_2of4

Index(['T002', 'T090', 'T095', 'T158', 'T028', 'T003', 'T114', 'T115', 'T151',
       'T039', 'T031', 'T073', 'T011', 'T225', 'T900', 'T013', 'T230', 'T008',
       'T221', 'T009', 'T102', 'T081', 'T082', 'T083', 'T089', 'T094', 'T220',
       'T128', 'T135', 'T228', 'T152', 'T080', 'T159', 'T223', 'T160', 'T155',
       'T903', 'T035', 'T065', 'T048', 'T027', 'T061', 'T012', 'T063', 'T066',
       'T033'],
      dtype='object', name='trap')

# Across years: no of positive traps across years

Across years: wnvpresent 
    > wnvpresent 8 times (17 traps) ay_8nabov
    > wnvpresent 6 times (25 traps) ay_6nabov

In [16]:
ay_c = betw_year.groupby(['trap'])['wnvpresent',].sum()\
.sort_values('wnvpresent', ascending=False)

ay_c.head()

,wnvpresent
trap,
T900,29.0
T002,15.0
T115,15.0
T003,14.0
T225,11.0


In [17]:
ay_8nabov = ay_c.loc[ay['wnvpresent']>7,:].index
ay_8nabov

Index(['T900', 'T002', 'T115', 'T003', 'T225', 'T011', 'T013', 'T028', 'T223',
       'T138', 'T114', 'T903', 'T008', 'T009', 'T061', 'T082', 'T135'],
      dtype='object', name='trap')

In [18]:
ay_6nabov = ay_c.loc[ay['wnvpresent']>5,:].index
ay_6nabov

Index(['T900', 'T002', 'T115', 'T003', 'T225', 'T011', 'T013', 'T028', 'T223',
       'T138', 'T114', 'T903', 'T008', 'T009', 'T061', 'T082', 'T135', 'T027',
       'T030', 'T090', 'T235', 'T086', 'T230', 'T221', 'T128', 'T066', 'T151',
       'T155', 'T228', 'T233'],
      dtype='object', name='trap')

# Within years: wnvpresent (at least 4)

Within years: wnvpresent at least 4
2007 (19 traps) wy_2007_4nabv
2009 (0 traps)  0
2011 (3 traps)  wy_2011_4nabv
2013 (22 traps) wy_2013_4nabv

In [19]:
wy_2007_4nabv = betw_year.loc[(betw_year['year']==2007) & (betw_year['wnvcount']>3), 'trap']
wy_2009_4nabv = betw_year.loc[(betw_year['year']==2009) & (betw_year['wnvcount']>3), 'trap']
wy_2011_4nabv = betw_year.loc[(betw_year['year']==2011) & (betw_year['wnvcount']>3), 'trap']
wy_2013_4nabv = betw_year.loc[(betw_year['year']==2013) & (betw_year['wnvcount']>3), 'trap']

# Within years: wnvpresent (at least 4)

Within years: wnvpresent at least 3
2007 (27 traps) wy_2007_3nabv
2009 (0 traps)  0
2011 (4 traps)  wy_2011_3nabv
2013 (33 traps) wy_2013_3nabv

In [20]:
wy_2007_3nabv = betw_year.loc[(betw_year['year']==2007) & (betw_year['wnvcount']>2), 'trap']
wy_2009_3nabv = betw_year.loc[(betw_year['year']==2009) & (betw_year['wnvcount']>2), 'trap']
wy_2011_3nabv = betw_year.loc[(betw_year['year']==2011) & (betw_year['wnvcount']>2), 'trap']
wy_2013_3nabv = betw_year.loc[(betw_year['year']==2013) & (betw_year['wnvcount']>2), 'trap']

# Create groups

In [21]:
group_1 = list(ay_8nabov)
group_1.extend(ay_3of4)
group_1.extend(wy_2007_4nabv)
group_1.extend(wy_2011_4nabv)
group_1.extend(wy_2013_4nabv)
len(list(set(group_1)))

46

In [22]:
group_2 = list(ay_6nabov)
group_2.extend(ay_3of4)
group_2.extend(wy_2007_3nabv)
group_2.extend(wy_2011_3nabv)
group_2.extend(wy_2013_3nabv)
len(list(set(group_2)))

57

In [23]:
group_3 = list(ay_6nabov)
group_3.extend(ay_2of4)
group_3.extend(wy_2007_3nabv)
group_3.extend(wy_2011_3nabv)
group_3.extend(wy_2013_3nabv)
len(list(set(group_3)))

68

# ADD new df here
to merge on
latitude longtitude
species
date
trap

In [24]:
# read in data
train_merged = pd.read_csv('../data/merged_train.csv')

In [25]:
train_merged.columns

Index(['date', 'species', 'latitude', 'longitude', 'trap', 'nummosquitos',
       'wnvpresent', 'wnvcount', 'station', 'tmax', 'tmin', 'tavg', 'dewpoint',
       'wetbulb', 'heat', 'cool', 'codesum', 'preciptotal', 'stnpressure',
       'sealevel', 'resultspeed', 'resultdir', 'avgspeed', 'month', 'sunrise',
       'sunset', 'bcfg', 'br', 'dz', 'fg', 'fg+', 'fu', 'hz', 'mifg', 'ra',
       'sn', 'sq', 'ts', 'tsra', 'vcts', 'gr', 'vcfg', 'sprayed'],
      dtype='object')

In [26]:
# extract required columns
train_fe = train_merged[['date', 'species', 'latitude',
                         'longitude', 'trap', 'wnvpresent']].copy()

In [27]:
# from the previous EDA we understand that 
# certain species are more likely to have wnv
# certain weeks of the month are more likely to have wnv

In [28]:
# Custom dummy coding for species
train_fe['CULEX PIPIENS/RESTUANS'] = train_fe['species'].map(lambda x: 1 if x == 'CULEX PIPIENS/RESTUANS' else 0)
train_fe['CULEX PIPIENS'] = train_fe['species'].map(lambda x: 1 if x == 'CULEX PIPIENS' else 0)
train_fe['CULEX RESTUANS'] = train_fe['species'].map(lambda x: 1 if x == 'CULEX RESTUANS' else 0)

In [29]:
# create week variable
train_fe['date'] = pd.to_datetime(train_fe['date'])
train_fe['week'] = train_fe['date'].dt.isocalendar().week

In [30]:
# Custom dummy coding for week

train_fe.groupby('week')['wnvpresent'].sum()

week
22      0.0
23      0.0
24      0.0
25      0.0
26      1.0
27      0.0
28      7.0
29     15.0
30     16.0
31     39.0
32     54.0
33     66.0
34    107.0
35     47.0
36     44.0
37     34.0
38     19.0
39      6.0
40      2.0
41      0.0
Name: wnvpresent, dtype: float64

In [31]:
train_fe = pd.get_dummies(train_fe, columns=['week'], drop_first=False)

train_fe.drop(columns=['week_22', 'week_23', 'week_24', 'week_25', 
                            'week_26', 'week_27', 'week_40', 'week_41'],
                  axis=1,
                  inplace=True)

In [32]:
train_fe['trap_d'] = train_fe['trap']
train_fe = pd.get_dummies(train_fe, columns=['trap'])
train_fe.rename(columns = {'trap_d':'trap'}, inplace = True)
col_wo_trap = train_fe.columns[:53]

In [33]:
g1 = ['trap_' + x for x in group_1]
g1_others = list(col_wo_trap)
g1_others.extend(g1)
train_fe_g1 = train_fe[g1_others]
train_fe_g1.shape

(8475, 136)

In [34]:
g2 = ['trap_' + x for x in group_2]
g2_others = list(col_wo_trap)
g2_others.extend(g2)
train_fe_g2 = train_fe[g2_others]
train_fe_g2.shape

(8475, 170)

In [35]:
g3 = ['trap_' + x for x in group_3]
g3_others = list(col_wo_trap)
g3_others.extend(g3)
train_fe_g3 = train_fe[g3_others]
train_fe_g3.shape

(8475, 196)

# Merge

In [40]:
lag2.shape

(124768, 42)

In [41]:
train_fe_g1.shape

(8475, 136)

In [43]:
lag2['date'] = pd.to_datetime(lag2['date'])
lag2_g1 = lag2.copy()

lag2_g1 = lag2_g1.merge(train_fe_g1, how='left',
                             on=['date', 'species', 'latitude',
                                 'longitude', 'wnvpresent', 'trap'],
                            )

In [44]:
lag2_g1.shape

(124768, 172)

In [49]:
weather_lag_df = rf_weather_lag(lag0, 0)
weather_lag_df = rf_weather_lag(lag2, 2, weather_lag_df)

In [50]:
weather_lag_df

,weather_lag,train_acc,test_acc,sensitivity,specificity,roc_auc_score
0,0,92.94,93.44,16.67,97.51,77.03
1,2,93.00,93.30,14.04,97.51,76.51


In [80]:
weather_lag_df = rf_weather_lag(lag2_g1, 2, weather_lag_df)

In [71]:
df = lag2.copy()

In [53]:
df[[col for col in df.columns if (df[col].dtypes !='O') & 
                      (col not in ['date','block','addressaccuracy','nummosquitos','wnvcount','wnvpresent', 'station','species'])]]

,latitude,longitude,tmax,tmin,tavg,dewpoint,wetbulb,heat,cool,preciptotal,...,hz,mifg,ra,sn,sq,ts,tsra,vcts,gr,vcfg
0,41.867108,-87.654224,75,59,67.0,49,57.0,0.0,2.0,0.080,...,0,0.0,1,0,0,0,0,0,0.0,0.0
1,41.867108,-87.654224,75,59,67.0,49,57.0,0.0,2.0,0.080,...,0,0.0,1,0,0,0,0,0,0.0,0.0
2,41.862292,-87.648860,75,59,67.0,49,57.0,0.0,2.0,0.080,...,0,0.0,1,0,0,0,0,0,0.0,0.0
3,41.896282,-87.655232,75,59,67.0,49,57.0,0.0,2.0,0.080,...,0,0.0,1,0,0,0,0,0,0.0,0.0
4,41.907645,-87.760886,75,59,67.0,49,57.0,0.0,2.0,0.080,...,0,0.0,1,0,0,0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124763,41.925652,-87.633590,61,51,56.0,49,52.0,9.0,0.0,0.005,...,0,0.0,0,0,0,0,0,0,0.0,0.0
124764,41.925652,-87.633590,61,51,56.0,49,52.0,9.0,0.0,0.005,...,0,0.0,0,0,0,0,0,0,0.0,0.0
124765,41.925652,-87.633590,61,51,56.0,49,52.0,9.0,0.0,0.005,...,0,0.0,0,0,0,0,0,0,0.0,0.0
124766,41.925652,-87.633590,61,51,56.0,49,52.0,9.0,0.0,0.005,...,0,0.0,0,0,0,0,0,0,0.0,0.0


In [73]:
df = lag2_g1.copy()

In [61]:
lag2.dtypes.value_counts()

float64           19
int64             19
object             3
datetime64[ns]     1
dtype: int64

In [60]:
lag2_g1.dtypes.value_counts()

float64           149
int64              19
object              3
datetime64[ns]      1
dtype: int64

In [ ]:
df[col].dtypes

In [74]:
[df[col].dtypes for col in df.columns]

[dtype('<M8[ns]'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('float64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('flo

In [81]:
#lag2_test.iloc[:,60:80].info()

In [64]:
#weather_lag_df2 = rf_2_weather_lag(lag0, 0)

In [112]:
# weather_lag_df2 = rf_2_weather_lag(lag2_test, 2, weather_lag_df2)

In [67]:
weather_lag_df2

,weather_lag,train_acc,test_acc,sensitivity,specificity,roc_auc_score
0,0,79.28,88.53,71.05,80.75,85.06
1,2,79.78,88.16,71.93,81.10,84.54
2,3,78.82,87.64,75.44,80.20,85.07


In [ ]:
0
2
3
5
7
14

In [ ]:
weather_lag_df = rf_weather_lag(lag0, 0)
weather_lag_df = rf_weather_lag(lag2, 2, weather_lag_df)
weather_lag_df = rf_weather_lag(lag3, 3, weather_lag_df)
weather_lag_df = rf_weather_lag(lag5, 5, weather_lag_df)
weather_lag_df = rf_weather_lag(lag7, 7, weather_lag_df)
weather_lag_df = rf_weather_lag(lag14, 14, weather_lag_df)
weather_lag_df = rf_weather_lag(lag21, 21, weather_lag_df)
weather_lag_df = rf_weather_lag(lag28, 28, weather_lag_df)

In [118]:
weather_lag_df = rf_weather_lag(lag7, 7, weather_lag_df)

In [119]:
weather_lag_df

,weather_lag,train_acc,test_acc,sensitivity,specificity,roc_auc_score
0,0,93.06,93.20,21.05,97.31,75.99
1,7,92.79,93.35,15.79,97.26,76.30


In [104]:
ert = lag7_test.drop(columns='date', axis=1)

In [114]:
# ert[[col for col in ert.columns if (ert[col].dtypes !='O')]]

In [117]:
lag7.dtypes

date            datetime64[ns]
species                 object
latitude               float64
longitude              float64
trap                    object
nummosquitos           float64
wnvpresent             float64
wnvcount               float64
station                  int64
tmax                     int64
tmin                     int64
tavg                   float64
dewpoint                 int64
wetbulb                float64
heat                   float64
cool                   float64
codesum                 object
preciptotal            float64
stnpressure            float64
sealevel               float64
resultspeed            float64
resultdir                int64
avgspeed               float64
month                    int64
sunrise                float64
sunset                 float64
bcfg                     int64
br                       int64
dz                       int64
fg                       int64
fg+                      int64
fu                       int64
hz      

In [124]:
lag7.shape

(124768, 42)

In [128]:
type(lag7_test)

pandas.core.frame.DataFrame

In [133]:
df = lag7_test.copy()

In [134]:
df[[col for col in df.columns if (df[col].dtypes !='O') & 
                      (col not in ['date','block','addressaccuracy','nummosquitos','wnvcount','wnvpresent', 'station','species'])]]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [136]:
df['date'].dtypes

dtype('<M8[ns]')

In [137]:
lag7_test['date'].dtypes

dtype('<M8[ns]')

# Feature engineering for others

In [82]:
# read in data

train_merged = pd.read_csv('../data/merged_train.csv')

In [83]:
# to merge with weather data above
# train_merged = train_merged['date', 'latitude', 'longitude', 'trap']

# for interim analysis
train_weather = train_merged.drop(['nummosquitos','wnvcount',
                                   'station', 'sprayed'],
                                  axis=1)

In [84]:
train_weather.columns

Index(['date', 'species', 'latitude', 'longitude', 'trap', 'wnvpresent',
       'tmax', 'tmin', 'tavg', 'dewpoint', 'wetbulb', 'heat', 'cool',
       'codesum', 'preciptotal', 'stnpressure', 'sealevel', 'resultspeed',
       'resultdir', 'avgspeed', 'month', 'sunrise', 'sunset', 'bcfg', 'br',
       'dz', 'fg', 'fg+', 'fu', 'hz', 'mifg', 'ra', 'sn', 'sq', 'ts', 'tsra',
       'vcts', 'gr', 'vcfg'],
      dtype='object')

In [85]:
train_weather.head()

,date,species,latitude,longitude,trap,wnvpresent,tmax,tmin,tavg,dewpoint,...,hz,mifg,ra,sn,sq,ts,tsra,vcts,gr,vcfg
0,2007-05-29,CULEX PIPIENS/RESTUANS,41.867108,-87.654224,T048,0.0,88,65,76.5,59,...,1,0.0,0,0,0,0,0,0,0.0,0.0
1,2009-09-25,CULEX PIPIENS,41.776156,-87.778927,T155,0.0,70,60,65.0,57,...,0,0.0,1,0,0,0,0,0,0.0,0.0
2,2009-09-25,CULEX RESTUANS,41.923738,-87.785288,T013,1.0,70,60,65.0,57,...,0,0.0,1,0,0,0,0,0,0.0,0.0
3,2009-09-25,CULEX RESTUANS,41.960616,-87.777189,T017,0.0,70,60,65.0,57,...,0,0.0,1,0,0,0,0,0,0.0,0.0
4,2007-08-24,CULEX RESTUANS,41.662014,-87.724608,T135,0.0,81,69,75.0,67,...,0,0.0,1,0,0,1,1,0,0.0,0.0


In [86]:
# Custom dummy coding for species
train_weather['CULEX PIPIENS/RESTUANS'] = train_weather['species'].map(lambda x: 1 if x == 'CULEX PIPIENS/RESTUANS' else 0)
train_weather['CULEX PIPIENS'] = train_weather['species'].map(lambda x: 1 if x == 'CULEX PIPIENS' else 0)
train_weather['CULEX RESTUANS'] = train_weather['species'].map(lambda x: 1 if x == 'CULEX RESTUANS' else 0)

In [14]:
# create week variable
train_weather['date'] = pd.to_datetime(train_weather['date'])
train_weather['week'] = train_weather['date'].dt.isocalendar().week

In [15]:
# Custom dummy coding for week

train_weather.groupby('week')['wnvpresent'].sum()

week
22      0.0
23      0.0
24      0.0
25      0.0
26      1.0
27      0.0
28      7.0
29     15.0
30     16.0
31     39.0
32     54.0
33     66.0
34    107.0
35     47.0
36     44.0
37     34.0
38     19.0
39      6.0
40      2.0
41      0.0
Name: wnvpresent, dtype: float64

In [16]:
train_weather = pd.get_dummies(train_weather, columns=['week'], drop_first=False)

In [17]:
train_weather.columns

Index(['date', 'species', 'latitude', 'longitude', 'trap', 'wnvpresent',
       'tmax', 'tmin', 'tavg', 'dewpoint', 'wetbulb', 'heat', 'cool',
       'codesum', 'preciptotal', 'stnpressure', 'sealevel', 'resultspeed',
       'resultdir', 'avgspeed', 'month', 'sunrise', 'sunset', 'bcfg', 'br',
       'dz', 'fg', 'fg+', 'fu', 'hz', 'mifg', 'ra', 'sn', 'sq', 'ts', 'tsra',
       'vcts', 'gr', 'vcfg', 'CULEX PIPIENS/RESTUANS', 'CULEX PIPIENS',
       'CULEX RESTUANS', 'week_22', 'week_23', 'week_24', 'week_25', 'week_26',
       'week_27', 'week_28', 'week_29', 'week_30', 'week_31', 'week_32',
       'week_33', 'week_34', 'week_35', 'week_36', 'week_37', 'week_38',
       'week_39', 'week_40', 'week_41'],
      dtype='object')

In [18]:
train_weather.drop(columns=['week_22', 'week_23', 'week_24', 'week_25', 
                            'week_26', 'week_27', 'week_40', 'week_41'],
                  axis=1,
                  inplace=True)

In [138]:
lag7 = pd.read_csv(r'../data/weather_cleaned_lag7.csv')

# lag3

In [95]:
lag3.shape

(124768, 42)

In [96]:
lag3_ = lag3.copy()
lag3_['date'] = pd.to_datetime(lag3_['date'])

lag3_g1 = lag3_.merge(train_fe_g1, how='inner',
                    on=['date', 'species', 'latitude',
                        'longitude', 'wnvpresent', 'trap'],
                   )

In [97]:
lag3_g1.shape

(8475, 172)

## lag3_g1

In [98]:
lag3_g1 = lag3_g1.drop(columns=['trap', 'date', 'species', 'latitude', 'longitude',
                              'nummosquitos', 'wnvcount', 'station', 'codesum'], axis=1)

lag3_g1.shape

(8475, 163)

In [99]:
lag3_g1.to_csv('../data/lag3_g1.csv', index=False)

In [94]:
lag3_g1.head()

,wnvpresent,tmax,tmin,tavg,dewpoint,wetbulb,heat,cool,codesum,preciptotal,...,trap_T230,trap_T008,trap_T009,trap_T061,trap_T147,trap_T223,trap_T226,trap_T227,trap_T231,trap_T115
0,0.0,70,50,60.0,55,57.0,5.0,0.0,TSRA RA BR,0.28,...,0,0,0,0,0,0,0,0,0,0
1,0.0,70,50,60.0,55,57.0,5.0,0.0,TSRA RA BR,0.28,...,0,0,0,0,0,0,0,0,0,0
2,0.0,70,50,60.0,55,57.0,5.0,0.0,TSRA RA BR,0.28,...,0,0,0,0,0,0,0,0,0,0
3,0.0,70,50,60.0,55,57.0,5.0,0.0,TSRA RA BR,0.28,...,0,0,0,0,0,0,0,0,0,0
4,0.0,70,50,60.0,55,57.0,5.0,0.0,TSRA RA BR,0.28,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
## lag3_g2

In [101]:
lag3_ = lag3.copy()
lag3_['date'] = pd.to_datetime(lag3_['date'])

lag3_g2 = lag3_.merge(train_fe_g2, how='inner',
                    on=['date', 'species', 'latitude',
                        'longitude', 'wnvpresent', 'trap'],
                   )
lag3_g2.shape

(8475, 206)

In [103]:
lag3_g2 = lag3_g2.drop(columns=['trap', 'date', 'species', 'latitude', 'longitude',
                              'nummosquitos', 'wnvcount', 'station'], axis=1)

lag3_g2.shape

(8475, 198)

In [104]:
lag3_g2.to_csv('../data/lag3_g2.csv', index=False)

In [ ]:
## lag3_g2

In [105]:
lag3_ = lag3.copy()
lag3_['date'] = pd.to_datetime(lag3_['date'])

lag3_g3 = lag3_.merge(train_fe_g3, how='inner',
                    on=['date', 'species', 'latitude',
                        'longitude', 'wnvpresent', 'trap'],
                   )
lag3_g3.shape

(8475, 232)

In [106]:
lag3_g3 = lag3_g3.drop(columns=['trap', 'date', 'species', 'latitude', 'longitude',
                              'nummosquitos', 'wnvcount', 'station'], axis=1)

lag3_g3.shape

(8475, 224)

In [107]:
lag3_g3.to_csv('../data/lag3_g3.csv', index=False)

# Archive may delete

In [19]:
# To check if all the traps in test are also in train set

train_loc = []
for x, y in zip(train['longitude'], train['latitude']):
    if [x, y] not in train_loc:
        train_loc.append([x, y])
    
test_loc = []
for x, y in zip(test['longitude'], test['latitude']):
    if [x, y] not in test_loc:
        test_loc.append([x, y])
        
ex_loc = []
for i in test_loc:
    if i not in train_loc:
        ex_loc.append(i)

NameError: name 'train' is not defined

In [ ]:
train_merged.groupby(['latitude', 'longitude'])['nummosquitos'].sum()

In [ ]:
train_merged.groupby(['trap'])['nummosquitos'].sum()